In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv('datasets/q3_data.csv')

In [3]:
df

,Income,Credit,Label
0,Low,Good,Yes
1,Low,Bad,No
2,Medium,Good,Yes
3,Medium,Bad,Yes
4,High,Good,Yes
5,High,Bad,No


In [4]:
class Node:
    def __init__(self, feature):
        self.feature = feature
        self.children = {}
        self.dTree = None
        self.treshold = None

class DecisionTree:
    def __init__(self, predCollabel='label', dTreeType='id3'):
        self.predCollabel = predCollabel
        self.dTreeType = dTreeType

    def gini(self, data, feature=None):
        if not feature:
            feature = self.predCollabel
        total = len(data)
        if total == 0:
            return 0
        counts = data[feature].value_counts()
        gini = 0
        for count in counts:
            p = count / total
            gini += p ** 2
        return 1 - gini
    
    def gini_index_helper(self, df, feature):
        values = df[feature].unique()
        weighted_gini = 0
        for val in values:
            subset = df[df[feature] == val]
            weighted_gini += (len(subset) / len(df)) * self.gini(subset)
        return weighted_gini
    
    def gini_index(self, df, feature):
        if df[feature].dtype == 'int64':
            vals = sorted(set(df[feature].values.tolist()))
            infos = []
            for thr in vals[1:]:
                df_sub = df.copy()
                df_sub[feature+'Group'] = df_sub[feature].apply(lambda x: '<=t' if x <= thr else '>t')
                infos.append(-self.gini_index_helper(df_sub, feature+'Group'))
            
            if not infos:
                return 0.0, None
            split_idx = np.argmax(infos)

            return infos[split_idx], vals[split_idx+1]
        return -self.gini_index_helper(df, feature), None

    
    def entropy(self, data, feature=None):
        if not feature:
            feature = self.predCollabel
        total = len(data)
        if total == 0:
            return 0
        counts = data[feature].value_counts()
        entropy = 0
        for count in counts:
            p = count / total
            entropy -= p * math.log2(p)
        return entropy
    
    def info_gain(self, df, feature):
        if df[feature].dtype == 'int64':
            vals = sorted(set(df[feature].values.tolist()))
            infos = []
            for thr in vals[1:]:
                df_sub = df.copy()
                df_sub[feature+'Group'] = df_sub[feature].apply(lambda x: '<=t' if x <= thr else '>t')
                infos.append(info_gain_help(df_sub, feature+'Group'))
            
            if not infos:
                return 0.0, None
            split_idx = np.argmax(infos)

            return infos[split_idx], vals[split_idx+1]
        return self.info_gain_helper(df, feature), None
    
    def info_gain_helper(self, df, feature):
        total_entropy = self.entropy(df)
        values = df[feature].unique()
        weighted_entropy = 0
        for val in values:
            subset = df[df[feature] == val]
            weighted_entropy += (len(subset) / len(df)) * self.entropy(subset)
        return total_entropy - weighted_entropy
    
    def gain_ratio(self, data, feature):
        a = self.info_gain(data, feature)
        b = self.entropy(data, feature=feature)
        return a/b if b > 0 else 0
    
    def recursion(self, data, cols):
        if len(cols) == 0:
            return Node(data[self.predCollabel].value_counts().idxmax())
        
        info_gains, treshold = None, None
        if self.dTreeType == 'id3':
            arr = [self.info_gain(data, col) for col in cols]
            info_gains, treshold = np.hsplit(np.array(arr), 2)
        elif self.dTreeType == 'c4.5':
            info_gains, treshold = [], []
            for col in cols:
                a, tres = self.info_gain(data, col)
                b = self.entropy(data, feature=col)
                info_gains.append(a/b if b>0 else 0)
                treshold.append([tres])
        elif self.dTreeType == 'cart':
            arr = [self.gini_index(data, col) for col in cols]
            info_gains, treshold = np.hsplit(np.array(arr), 2)
        ftr_idx = np.argmax(info_gains)
        feature = cols[ftr_idx]
        tree = Node(feature)
        tree.treshold = treshold[ftr_idx][0]

        # Children nodes
        cols.remove(feature)
        if tree.treshold:
            df_child = df[df[feature] <= tree.treshold]
            tree.children['<=t'] = self.recursion(df_child, cols.copy())

            df_child = df[df[feature] > tree.treshold]
            tree.children['>t'] = self.recursion(df_child, cols.copy())
        else:
            for child in data[feature].unique():
                df_child = df[df[feature] == child]
                tree.children[child] = self.recursion(df_child ,cols.copy())

        return tree
    
    def fit(self, df):
        cols = df.columns.values.tolist()
        cols.remove(self.predCollabel)
        self.dTree = self.recursion(df, cols)
    
    def predict(self, data):
        predictions = []
        for row in data:
            tree = self.dTree
            while True:
                if len(tree.children) == 0:
                    predictions.append(tree.feature)
                    break
                feature = tree.feature
                tres = tree.treshold
                if tres:
                    label = '<=t' if row[feature] <= tres else '>t'
                    tree = tree.children[label]
                else:
                    tree = tree.children[row[feature]]
        return predictions

In [5]:
dtree = DecisionTree(predCollabel='Label', dTreeType='cart')
dtree.fit(df)

In [7]:
dtree.predict([{'Income':'Low', 'Credit': 'Good'}])

['Yes']